Our goal here is to find an optimal trend following strategy to trade cryptocurrencies. The process is as follows:

1) Start with an extremely simple strategy
2) Evaluate how it performs
3) Based on how it performs, add a bit more complexity to improve performance
4) Iterate



The first strategy we'll explore is extremely simple, and based on two indicators:
1) The Simple Moving Average (SMA)
2) The Exponential Moving Average (EMA)

The signals are generated as follows:
- If the current price of the crypto is above BOTH its SMA and EMA, we buy the crypto.
- If the current price of the crypto is below EITHER its SMA OR EMA, we sell the crypto.

It is a long-only strategy. We only sell the crypto if we were already holding it.

We will train one hyperparameter for this strategy: the window of the moving averages. We could also decide to train one window for the SMA and one window for the EMA, but for now we will only use the same for both for simplicity, and add complexity if necessary.

In [1]:
import pandas as pd
import numpy as np
from data_functions import *
from backtest_functions import *

btc = download_crypto_data('bitcoin')

Data successfully saved to bitcoin.csv


In [2]:
def train_strategy(train_data, price_col, strategy_col, possible_windows):
    return grid_search(train_data, price_col, strategy_col, window_range=possible_windows)

def test_strategy(test_data, price_col, strategy_col, window_size):
    # Implement your testing logic here
    # Return performance metrics
    # Example: return {'profit': 1000, 'sharpe_ratio': 1.5}
    trend_test = trend_following(test_data, price_col, strategy_col, window_size)
    total_perf, cagr, _, sharpe_ratio, max_drawdown = return_metrics(trend_test, price_col, strategy_col)
    return {'Strategy_Total_Performance': total_perf, 'Strategy_Max_Drawdown': max_drawdown}

def buy_and_hold_return(test_data, price_col):
    start_price = test_data.iloc[0][price_col]  # Price at the start of the test period
    end_price = test_data.iloc[-1][price_col]  # Price at the end of the test period
    return (end_price - start_price) / start_price  # Return in percentage

def cross_validate_strategy(data, price_col, strategy_col, splits, possible_windows):
    results = []

    for split in splits:
        train_start, train_end, test_start, test_end = split
        train_data = data[(data.index >= train_start) & (data.index <= train_end)]
        test_data = data[(data.index >= test_start) & (data.index <= test_end)]
        
        buy_hold_return = buy_and_hold_return(test_data, price_col)
        test_data['buy_and_hold_return'] = test_data[price_col].pct_change()
        buy_hold_dd = max_drawdown(test_data, 'buy_and_hold_return')


        # Find the best window in the training data
        best_window = train_strategy(train_data, price_col, strategy_col, possible_windows)
        #best_window = 122
        
        # Test this window in the test data
        test_result = test_strategy(test_data, price_col, strategy_col, best_window)

        results.append({
            'train_period': f"{train_start} to {train_end}",
            'test_period': f"{test_start} to {test_end}",
            'best_window': best_window,
            'Buy_Hold_Performance': buy_hold_return,
            'Buy_Hold_Max_Drawdown': buy_hold_dd,
            **test_result  # This unpacks the test_result dictionary into key-value pairs
        })

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

In [3]:
# Example Usage
bitcoin_data = pd.read_csv('bitcoin.csv')
bitcoin_data.set_index('Timestamp', inplace=True)
bitcoin_data.index = pd.to_datetime(bitcoin_data.index)
splits = [
    ('2013-12-31', '2015-12-31', '2015-12-31', '2017-12-31'),
    ('2015-12-31', '2017-12-31', '2017-12-31', '2019-12-31'),
    ('2017-12-31', '2019-12-31', '2019-12-31', '2021-12-31'),
    ('2019-12-31', '2021-12-31', '2021-12-31', '2023-12-31')
]
possible_windows = range(5, 300)  # Example range of window sizes

In [4]:
%%time
df=cross_validate_strategy(data=bitcoin_data, price_col='bitcoin', 
                        strategy_col='strategy_return', splits=splits, possible_windows=possible_windows)

/var/folders/m_/7p6jl6417z14r0fws5vfggx40000gn/T/ipykernel_2391/2087316758.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['buy_and_hold_return'] = test_data[price_col].pct_change()
/Users/allanbellahsene/Desktop/ALGO_TRADING/backtest_functions.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'cumulative_return_{strategy_col}'] = (1+df[f'{strategy_col}']).cumprod() - 1
/Users/allanbellahsene/Desktop/ALGO_TRADING/backtest_functions.py:52: SettingWithCopyWarning: 
A value is trying to b

CPU times: user 22.3 s, sys: 192 ms, total: 22.4 s
Wall time: 26.7 s


In [5]:
df

,train_period,test_period,best_window,Buy_Hold_Performance,Buy_Hold_Max_Drawdown,Strategy_Total_Performance,Strategy_Max_Drawdown
0,2013-12-31 to 2015-12-31,2015-12-31 to 2017-12-31,11,33.419505,0.362421,18.725983,0.235917
1,2015-12-31 to 2017-12-31,2017-12-31 to 2019-12-31,5,-0.512087,0.824646,0.387616,0.384538
2,2017-12-31 to 2019-12-31,2019-12-31 to 2021-12-31,20,5.517828,0.528571,3.311883,0.446601
3,2019-12-31 to 2021-12-31,2021-12-31 to 2023-12-31,41,-0.105341,0.670771,0.372463,0.392739


How the hell did I find a window of 122 to be the best strategy?